# Lab5: Training & hyperparameter tuning TensorFlow

**Automate efficient hyperparameter tuning** by using Azure Machine Learning **HyperDrive package**. <br>

<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>

Learn how to complete the steps required to tune hyperparameters with the Azure Machine Learning SDK:

- Define the parameter search space
- Specify a primary metric to optimize
- Specify early termination policy for low-performing runs
- Allocate resources
- Launch an experiment with the defined configuration
- Visualize the training runs
- Select the best configuration for your model


**Hyperparameters are adjustable parameters** that let you control the model training process. For example, with neural networks, you decide the number of hidden layers and the number of nodes in each layer. Model performance depends heavily on hyperparameters.

Hyperparameter tuning is the process of finding the configuration of hyperparameters that results in the best performance. The process is typically computationally expensive and manual.

Azure Machine Learning lets you automate hyperparameter tuning and run experiments in parallel to efficiently optimize hyperparameters.

> Documentation: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters

This tutorial shows how to train a simple deep neural network using the MNIST dataset and TensorFlow on Azure Machine Learning. MNIST is a popular dataset consisting of 70,000 grayscale images. Each image is a handwritten digit of `28x28` pixels, representing number from 0 to 9. The goal is to create a multi-class classifier to identify the digit each image represents, and deploy it as a web service in Azure.

For more information about the MNIST dataset, please visit [Yann LeCun's website](http://yann.lecun.com/exdb/mnist/).

In [ ]:
import sys
sys.version

In [ ]:
import datetime
now = datetime.datetime.now()
print(now)

In [3]:
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt

In [4]:
import azureml
from azureml.core import Workspace

# check core SDK version number
print("You are using Azure ML version", azureml.core.VERSION)

You are using Azure ML version 1.18.0


## Experimentation

In [5]:
ws = Workspace.from_config()

In [6]:
from azureml.core import Experiment

script_folder = './sample_projects/lab5'
os.makedirs(script_folder, exist_ok=True)

exp = Experiment(workspace=ws, name='Lab5-HyperparameterTuning')

## Data

In [7]:
import urllib

os.makedirs('./data/mnist', exist_ok=True)

urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz', filename = './data/mnist/train-images.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', filename = './data/mnist/train-labels.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz', filename = './data/mnist/test-images.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', filename = './data/mnist/test-labels.gz')

('./data/mnist/test-labels.gz', <http.client.HTTPMessage at 0x7f27ee1245f8>)

In [8]:
from utils import load_data

# note we also shrink the intensity values (X) from 0-255 to 0-1. This helps the neural network converge faster.
X_train = load_data('./data/mnist/train-images.gz', False) / 255.0
y_train = load_data('./data/mnist/train-labels.gz', True).reshape(-1)

X_test = load_data('./data/mnist/test-images.gz', False) / 255.0
y_test = load_data('./data/mnist/test-labels.gz', True).reshape(-1)

count = 0
sample_size = 30
plt.figure(figsize = (16, 6))
for i in np.random.permutation(X_train.shape[0])[:sample_size]:
    count = count + 1
    plt.subplot(1, sample_size, count)
    plt.axhline('')
    plt.axvline('')
    plt.text(x = 10, y = -10, s = y_train[i], fontsize = 18)
    plt.imshow(X_train[i].reshape(28, 28), cmap = plt.cm.Greys)
plt.show()

## Create a Dataset for Files
A Dataset can reference single or multiple files in your datastores or public urls. The files can be of any format. Dataset provides you with the ability to download or mount the files to your compute. By creating a dataset, you create a reference to the data source location. If you applied any subsetting transformations to the dataset, they will be stored in the dataset as well. The data remains in its existing location, so no extra storage cost is incurred. [Learn More](https://aka.ms/azureml/howto/createdatasets)

In [9]:
from azureml.core.dataset import Dataset
web_paths = ['http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
            ]
dataset = Dataset.File.from_files(path = web_paths)

In [10]:
dataset = dataset.register(workspace = ws,
                           name = 'mnist dataset',
                           description='training and test dataset',
                           create_new_version=True)
# list the files referenced by dataset
dataset.to_path()

['/http%3A/%2Fyann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
 '/http%3A/%2Fyann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
 '/http%3A/%2Fyann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
 '/http%3A/%2Fyann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz']

## Azure ML Compute

If we could not find the cluster with the given name, then we will create a new cluster here. We will create an `AmlCompute` cluster of `STANDARD_NC6` GPU VMs. This process is broken down into 3 steps:
1. create the configuration (this step is local and only takes a second)
2. create the cluster (this step will take about **20 seconds**)
3. provision the VMs to bring the cluster to the initial size (of 1 in this case). This step will take about **3-5 minutes** and is providing only sparse output in the process. Please make sure to wait until the call returns before moving to the next cell

In [11]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, "(" , ct.type, ") :", ct.provisioning_state)

AzureDatabricks ( Databricks ) : Succeeded
aksinstanceml ( AKS ) : Succeeded
instancenotebooksr ( ComputeInstance ) : Succeeded
computeclustd12 ( AmlCompute ) : Succeeded
automl ( AmlCompute ) : Succeeded
designer ( AmlCompute ) : Succeeded
instancestdd14 ( AmlCompute ) : Succeeded
cpupipelines ( AmlCompute ) : Succeeded
CPUcluster ( AmlCompute ) : Succeeded
GPUclusterNC6 ( AmlCompute ) : Succeeded


### Let's create a GPU Azure ML cluster

In [12]:
%%time
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "GPUclusterNC6"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', # A GPU instance
                                                           min_nodes=0,
                                                           max_nodes=8,
                                                           idle_seconds_before_scaledown=14400)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
{'currentNodeCount': 3, 'targetNodeCount': 3, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 3, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-12-14T11:18:09.070000+00:00', 'errors': None, 'creationTime': '2020-12-14T11:04:31.170238+00:00', 'modifiedTime': '2020-12-14T11:04:46.984805+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 8, 'nodeIdleTimeBeforeScaleDown': 'PT14400S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}
CPU times: user 16.4 ms, sys: 4.12 ms, total: 20.5 ms
Wall time: 325 ms


In [13]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, "(" , ct.type, ") :", ct.provisioning_state)

AzureDatabricks ( Databricks ) : Succeeded
aksinstanceml ( AKS ) : Succeeded
instancenotebooksr ( ComputeInstance ) : Succeeded
computeclustd12 ( AmlCompute ) : Succeeded
automl ( AmlCompute ) : Succeeded
designer ( AmlCompute ) : Succeeded
instancestdd14 ( AmlCompute ) : Succeeded
cpupipelines ( AmlCompute ) : Succeeded
CPUcluster ( AmlCompute ) : Succeeded
GPUclusterNC6 ( AmlCompute ) : Succeeded


## Copy the training files into the script folder
The TensorFlow training script is already created for you. You can simply copy it into the script folder, together with the utility library used to load compressed data file into numpy array.

In [14]:
import shutil

# the training logic is in the tf_mnist.py file.
shutil.copy('./tf_mnist.py', script_folder)

# the utils.py just helps loading data from the downloaded MNIST dataset into numpy arrays.
shutil.copy('./utils.py', script_folder)

'./sample_projects/lab5/utils.py'

## Construct neural network in TensorFlow
In the training script `tf_mnist.py`, it creates a very simple DNN (deep neural network), with just 2 hidden layers. The input layer has 28 * 28 = 784 neurons, each representing a pixel in an image. The first hidden layer has 300 neurons, and the second hidden layer has 100 neurons. The output layer has 10 neurons, each representing a targeted label from 0 to 9.



In [15]:
with open(os.path.join(script_folder, './tf_mnist.py'), 'r') as f:
    print(f.read())

import numpy as np
import argparse
import os
import tensorflow as tf
import glob

from azureml.core import Run
from utils import load_data

print("TensorFlow version:", tf.__version__)

parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
parser.add_argument('--batch-size', type=int, dest='batch_size', default=50, help='mini batch size for training')
parser.add_argument('--first-layer-neurons', type=int, dest='n_hidden_1', default=100,
                    help='# of neurons in the first layer')
parser.add_argument('--second-layer-neurons', type=int, dest='n_hidden_2', default=100,
                    help='# of neurons in the second layer')
parser.add_argument('--learning-rate', type=float, dest='learning_rate', default=0.01, help='learning rate')
args = parser.parse_args()

data_folder = args.data_folder
print('Data folder:', data_folder)

# load train and test set into numpy arrays
# note we scale the

## TensorFlow estimator

Next, we construct an `azureml.train.dnn.TensorFlow` estimator object, use the Batch AI cluster as compute target, and pass the mount-point of the datastore to the training code as a parameter.

The TensorFlow estimator is providing a simple way of launching a TensorFlow training job on a compute target. It will automatically provide a docker image that has TensorFlow installed -- if additional pip or conda packages are required, their names can be passed in via the `pip_packages` and `conda_packages` arguments and they will be included in the resulting docker.

The TensorFlow estimator also takes a `framework_version` parameter -- if no version is provided, the estimator will default to the latest version supported by AzureML. Use `TensorFlow.get_supported_versions()` to get a list of all versions supported by your current SDK version or see the [SDK documentation](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.dnn?view=azure-ml-py) for the versions supported in the most current release.

In [16]:
tags = {"Framework" : "TensorFlow" , "BatchSize" : "50" , "H1" : "100" , "H2" : "50" , "LR" : "0.01"}

In [17]:
from azureml.train.dnn import TensorFlow

script_params = {
    '--data-folder': dataset.as_named_input('mnist').as_mount(),
    '--batch-size': 50,
    '--first-layer-neurons': 100,
    '--second-layer-neurons': 50,
    '--learning-rate': 0.01
}

est = TensorFlow(source_directory=script_folder,
                 script_params=script_params,
                 compute_target=compute_target,
                 entry_script='tf_mnist.py',
                 use_gpu=True,
                 pip_packages=['azureml-dataprep[pandas,fuse]'])

WARNING - framework_version is not specified, defaulting to version 1.13.


## Submit job to run
Submit the estimator to an Azure ML experiment to kick off the execution.

In [18]:
run = exp.submit(est, tags=tags)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.


### Monitor the Run <a class="anchor" id="monitor-run"></a>
As the Run is executed, it will go through the following stages:
1. Preparing: A docker image is created matching the Python environment specified by the TensorFlow estimator and it will be uploaded to the workspace's Azure Container Registry. This step will only happen once for each Python environment -- the container will then be cached for subsequent runs. Creating and uploading the image takes about **5 minutes**. While the job is preparing, logs are streamed to the run history and can be viewed to monitor the progress of the image creation.

2. Scaling: If the compute needs to be scaled up (i.e. the Batch AI cluster requires more nodes to execute the run than currently available), the cluster will attempt to scale up in order to make the required amount of nodes available. Scaling typically takes about **5 minutes**.

3. Running: All scripts in the script folder are uploaded to the compute target, data stores are mounted/copied and the `entry_script` is executed. While the job is running, stdout and the `./logs` folder are streamed to the run history and can be viewed to monitor the progress of the run.

4. Post-Processing: The `./outputs` folder of the run is copied over to the run history

There are multiple ways to check the progress of a running job. We can use a Jupyter notebook widget. 

> Note: The widget will automatically update ever 10-15 seconds, always showing you the most up-to-date information about the run

In [19]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [21]:
print("Run status =", run.get_status())

Run status = Completed


In [22]:
run.get_details()

{'runId': 'Lab5-HyperparameterTuning_1607954685_0a73b316',
 'target': 'GPUclusterNC6',
 'status': 'Completed',
 'startTimeUtc': '2020-12-14T14:04:58.217632Z',
 'endTimeUtc': '2020-12-14T14:06:16.944248Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '1cda4d8e-92fd-46dc-bc0a-b01df88de68d',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': 'ba55ec11-3b26-490e-b3a4-2f3cb61fb16e'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'mnist', 'mechanism': 'Mount'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'tf_mnist.py',
  'useAbsolutePath': False,
  'arguments': ['--data-folder',
   'DatasetConsumptionConfig:mnist',
   '--batch-size',
   '50',
   '--first-layer-neurons',
   '100',
   '--second-layer-neurons',
   '50',
   '--learning-rate',
   '0.01'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None'

In [23]:
run

Run(Experiment: Lab5-HyperparameterTuning,
Id: Lab5-HyperparameterTuning_1607954685_0a73b316,
Type: azureml.scriptrun,
Status: Completed)

### The Run object <a class="anchor" id="run-object"></a>
The Run object provides the interface to the run history -- both to the job and to the control plane (this notebook), and both while the job is running and after it has completed. It provides a number of interesting features for instance:
* `run.get_details()`: Provides a rich set of properties of the run
* `run.get_metrics()`: Provides a dictionary with all the metrics that were reported for the Run
* `run.get_file_names()`: List all the files that were uploaded to the run history for this Run. This will include the `outputs` and `logs` folder, azureml-logs and other logs, as well as files that were explicitly uploaded to the run using `run.upload_file()`

Below are some examples -- please run through them and inspect their output. 

In [24]:
run.get_metrics()

{'training_acc': [0.8199999928474426,
  0.9399999976158142,
  0.8999999761581421,
  0.9599999785423279,
  0.9599999785423279,
  0.9599999785423279,
  0.9200000166893005,
  0.9800000190734863,
  0.9599999785423279,
  1.0,
  0.9399999976158142,
  0.9399999976158142,
  0.9599999785423279,
  1.0,
  0.9800000190734863,
  1.0,
  0.9800000190734863,
  0.9399999976158142,
  1.0,
  1.0],
 'validation_acc': [0.8938999772071838,
  0.9150999784469604,
  0.925000011920929,
  0.9351999759674072,
  0.9384999871253967,
  0.9434000253677368,
  0.9465000033378601,
  0.9498000144958496,
  0.9520000219345093,
  0.9550999999046326,
  0.9567000269889832,
  0.9598000049591064,
  0.9598000049591064,
  0.9603999853134155,
  0.9627000093460083,
  0.9629999995231628,
  0.9652000069618225,
  0.9659000039100647,
  0.9668999910354614,
  0.9685999751091003],
 'final_acc': 0.9685999751091003}

## Plot accuracy over epochs
Since we can retrieve the metrics from the run, we can easily make plots using `matplotlib` in the notebook. Then we can add the plotted image to the run using `run.log_image()`, so all information about the run is kept together.

In [25]:
os.makedirs('./graphs', exist_ok=True)
metrics = run.get_metrics()

plt.figure(figsize = (13,5))
plt.plot(metrics['validation_acc'], 'r-', lw=4, alpha=.6)
plt.plot(metrics['training_acc'], 'b--', alpha=0.5)
plt.legend(['Full dataset', 'Mini-batch'])
plt.xlabel('epochs', fontsize=14)
plt.ylabel('accuracy', fontsize=14)
plt.title('Accuracy Epochs', fontsize=16)
run.log_image(name='TFgraph.png', plot=plt)
plt.show()

In [26]:
os.makedirs('./model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)

## Intelligent hyperparameter tuning with Azure HyperDrive

Hyperparameters are adjustable parameters you choose for model training that guide the training process. **The HyperDrive package** helps you automate choosing these parameters. <br>For example, you can define the parameter search space as discrete or continuous, and a sampling method over the search space as random, grid, or Bayesian. Also, you can specify a primary metric to optimize in the hyperparameter tuning experiment, and whether to minimize or maximize that metric. You can also define early termination policies in which poorly performing experiment runs are canceled and new ones started.

> Documentation: https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive?view=azure-ml-py

In [27]:
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform

ps = RandomParameterSampling(
    {
        '--batch-size': choice(25, 50, 100),
        '--first-layer-neurons': choice(10, 50, 100, 200, 300),
        '--second-layer-neurons': choice(10, 50, 100),
        '--learning-rate': loguniform(-6, -1)
    }
)

In [28]:
est = TensorFlow(source_directory=script_folder,
                 script_params={'--data-folder': dataset.as_named_input('mnist').as_mount()},
                 compute_target=compute_target,
                 entry_script='tf_mnist.py', 
                 use_gpu=True,
                 pip_packages=['azureml-dataprep[pandas,fuse]'])

WARNING - framework_version is not specified, defaulting to version 1.13.


Now we will define an early termnination policy. The `BanditPolicy` basically states to check the job every 2 iterations. If the primary metric (defined later) falls outside of the top 10% range, Azure ML terminate the job. This saves us from continuing to explore hyperparameters that don't show promise of helping reach our target metric. No Extra costs.

In [29]:
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

In [30]:
hypparmtuning = HyperDriveConfig(estimator=est, 
                       hyperparameter_sampling=ps, 
                       policy=policy, 
                       primary_metric_name='validation_acc', 
                       primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                       max_total_runs=6,
                       max_concurrent_runs=4)

In [31]:
tags = {"Framework" : "TensorFlow" , 
        "Hyperdrive" : "Yes"}

In [32]:
hypparmtuning = exp.submit(config=hypparmtuning, tags=tags)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.


### Let's run the hyperparameter tuning process:

In [33]:
RunDetails(hypparmtuning).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [34]:
#hypparmtuning.wait_for_completion(show_output=True)

### Checking the run:

In [41]:
print("Run status:", hypparmtuning.get_status())

Run status: Completed


### Results are available in the widget and in the experiment as well:

In [42]:
hypparmtuning

Run(Experiment: Lab5-HyperparameterTuning,
Id: HD_07064f72-5c23-4395-99fc-3932f25d9144,
Type: hyperdrive,
Status: Completed)

### We can save now the best model into the Azure ML model repository

In [43]:
best_run = hypparmtuning.get_best_run_by_primary_metric()

In [44]:
print(best_run.get_file_names())

['azureml-logs/55_azureml-execution-tvmps_2e4419f6d2e418f69d46db2992c2311fa734c4d4fcba8b8c746f59015bee9583_d.txt', 'azureml-logs/65_job_prep-tvmps_2e4419f6d2e418f69d46db2992c2311fa734c4d4fcba8b8c746f59015bee9583_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_2e4419f6d2e418f69d46db2992c2311fa734c4d4fcba8b8c746f59015bee9583_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/80_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_37b05554-4957-4f82-bba8-0828063dfcf1.jsonl', 'logs/azureml/dataprep/engine_spans_e9dc7408-81a9-4817-9bb5-facee09fbe50.jsonl', 'logs/azureml/dataprep/python_span_37b05554-4957-4f82-bba8-0828063dfcf1.jsonl', 'logs/azureml/dataprep/python_span_e9dc7408-81a9-4817-9bb5-facee09fbe50.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'logs/azureml/sidecar/tvmps_2e4419f6d

Registering the model:

In [45]:
model = best_run.register_model(model_name='TensorFlow', 
                                model_path='outputs/model', 
                                tags={'Framework':'TensorFlow', 
                                      'Hyperdrive':'Yes' , 
                                      'GPU':'Yes'})

Versions of the model:

In [46]:
from azureml.core.model import Model
models = Model.list(ws, name="TensorFlow")
for m in models:
    print(m.name, "(version =", m.version,") - Tags=",m.tags)

TensorFlow (version = 13 ) - Tags= {'Framework': 'TensorFlow', 'Hyperdrive': 'Yes', 'GPU': 'Yes'}
TensorFlow (version = 12 ) - Tags= {'Framework': 'TensorFlow', 'Hyperdrive': 'Yes', 'GPU': 'Yes'}
TensorFlow (version = 11 ) - Tags= {'Framework': 'TensorFlow', 'Hyperdrive': 'Yes', 'GPU': 'Yes'}
TensorFlow (version = 10 ) - Tags= {'Framework': 'TensorFlow', 'Hyperdrive': 'Yes', 'GPU': 'Yes'}
TensorFlow (version = 9 ) - Tags= {'Framework': 'TensorFlow', 'Hyperdrive': 'Yes', 'GPU': 'Yes'}
TensorFlow (version = 8 ) - Tags= {'Framework': 'TensorFlow', 'Hyperdrive': 'Yes', 'GPU': 'Yes'}
TensorFlow (version = 7 ) - Tags= {'Framework': 'TensorFlow', 'Hyperdrive': 'Yes', 'GPU': 'Yes'}
TensorFlow (version = 6 ) - Tags= {'Framework': 'TensorFlow', 'Hyperdrive': 'Yes', 'GPU': 'Yes'}
TensorFlow (version = 5 ) - Tags= {'Framework': 'TensorFlow', 'Hyperdrive': 'Yes', 'GPU': 'Yes'}
TensorFlow (version = 4 ) - Tags= {'Framework': 'TensorFlow', 'Hyperdrive': 'Yes', 'GPU': 'Yes'}
TensorFlow (version = 3 ) 

All models from the current workspace:

In [47]:
from azureml.core.model import Model
models = Model.list(ws)
for m in models:
    print(m.name, "( version:", m.version, ")")

TensorFlow ( version: 13 )
mybestmodel ( version: 12 )
mnist-prs ( version: 5 )
sklearn_mnist ( version: 18 )
IBM_attrition_explainer ( version: 13 )
local_deploy_model ( version: 13 )
DiabetesRidge ( version: 20 )
TensorFlow ( version: 12 )
mybestmodel ( version: 11 )
mnist-prs ( version: 4 )
sklearn_mnist ( version: 17 )
DiabetesRidge ( version: 19 )
IBM_attrition_explainer ( version: 12 )
local_deploy_model ( version: 12 )
ModeleLCL ( version: 1 )
DiabetesRidge ( version: 18 )
TensorFlow ( version: 11 )
mybestmodel ( version: 10 )
monmodeleaml ( version: 2 )
sklearn_mnist ( version: 16 )
DiabetesRidge ( version: 17 )
IBM_attrition_explainer ( version: 11 )
local_deploy_model ( version: 11 )
sklearn_mnist ( version: 15 )
TensorFlow ( version: 10 )
monmodeleaml ( version: 1 )
DiabetesRidge ( version: 16 )
DiabetesRidge ( version: 15 )
mnist-prs ( version: 3 )
TensorFlow ( version: 9 )
mybestmodel ( version: 9 )
sklearn_mnist ( version: 14 )
DiabetesRidge ( version: 14 )
IBM_attrition_

## We can delete now our GPU instance

In [48]:
#To delete the compute target
#compute_target.delete()

In [49]:
# Checking the deletion of the compute target
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, "(" , ct.type, ") :", ct.provisioning_state)

AzureDatabricks ( Databricks ) : Succeeded
aksinstanceml ( AKS ) : Succeeded
instancenotebooksr ( ComputeInstance ) : Succeeded
computeclustd12 ( AmlCompute ) : Succeeded
automl ( AmlCompute ) : Succeeded
designer ( AmlCompute ) : Succeeded
instancestdd14 ( AmlCompute ) : Succeeded
cpupipelines ( AmlCompute ) : Succeeded
CPUcluster ( AmlCompute ) : Succeeded
GPUclusterNC6 ( AmlCompute ) : Succeeded
monclusteraml ( AmlCompute ) : Succeeded


The compute instance will be deleted in 1 or 2 minutes.
You can check this using the Azure ML Studio as well.

> You can open the **Lab6 notebook**.

<img src="https://github.com/retkowsky/images/blob/master/Powered-by-MS-Azure-logo-v2.png?raw=true" height="300" width="300">